### Fine-tuning de un modelo preentrenado

El entrenamiento de un modelo partiendo de una inicialización aleatoria de los parámetros del mismo es una tarea muy costosa computacionalmente. Especialmente para modelos con arquitectura de tipo Transformers, donde los requerimientos computacionales pueden implicar el uso de cientos de GPUs durante semanas, e incluso más. Por ello, es más habitual en entornos modestos partir de una inicialización de los pesos procedente de un modelo preentrenado para una tarea más general que aquella que se quiere acometer. 

Este proceso, que tiene unos requerimientos computacionales mucho más modestos recibe el nombre de **fine-tuning**. En este caso, los pesos ya están inicializados para resolver una tarea más genérica, pero se expone durante varios ciclos de entrenamiento a un *dataset* más pequeño que el inicial, pero diseñado para que el modelo aprenda una tarea más específica. 

Este notebook va a demostrar cómo realizar dicho proceso utilizando la librería Transformers. Se va a realizar siguiendo dos vías diferentes.

La más sencilla es utilizar la clase Trainer de la librería transformers. 

Una vía un poco más compleja es utilizando un modelo preentrenado utilizando la librería Pytorch.

## Preparación del *dataset*

El primer paso para hacer fine-tuning de un modelo es descargar un dataset (o crear el nuestro propio) y prepararlo para el entrenamiento. El notebook anterior (04) nos enseñó varias técnicas para realizar este proceso utilizando la librería transformers.

In [ ]:
# Descargar el dataset de review de yelp
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")
# Imprimir un ejemplo del dataset
dataset["train"][100]
from transformers import AutoTokenizer
# Descargar el tokenizador de BERT
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
# Función para tokenizar los ejemplos, se aplica truncation y padding
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
# Aplicar la función a todo el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
# Utilizar un subconjunto del dataset reducido si no queremos entrenar con todo el dataset
# Si lo utilizamos todo, se tarda mucho en entrenar
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Found cached dataset parquet (/home/ulc/es/dac/.cache/huggingface/datasets/parquet/yelp_review_full-66f1f8c8d1a2da02/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 